<h1 style='text-align:center'>Titanic.</h1>
<br>


**Titanic** é um dos naufrágios mais infames da história. Em 15 de abril de 1912, durante sua viagem inaugural, o **Titanic** afundou após colidir com um iceberg, matando *1502* de *2224* passageiros e tripulantes. Essa tragédia sensacional chocou a comunidade internacional e levou a melhores regulamentações de segurança para navios.<br><br>

**O que precisamos fazer especificamente neste desafio?**

Nesse desafio, precisamos concluir a análise de que tipos de pessoas provavelmente sobreviveriam. Em particular, aplicamos as ferramentas de aprendizado de máquina para prever quais passageiros sobreviveram à tragédia?.

### Importando os dados

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#importa os dados de treino e teste
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
name=train.Name
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<h3>What does this data set mean.</h3>
____

Os dados foram divididos em dois grupos:
- conjunto de treinamento (train.csv)
- conjunto de teste (teste.csv)
<br>

O conjunto de treinamento inclui o status de sobrevivência dos passageiros (também conhecido como a verdade da tragédia titânica) que, juntamente com outros recursos, como gênero, classe, tarifa e pclass, é usado para criar o modelo de aprendizado de máquina.
<br><br>
O conjunto de teste deve ser usado para ver o desempenho do meu modelo em dados não vistos. O conjunto de teste não fornece o status de sobrevivência dos passageiros. Vamos usar nosso modelo para prever o status de sobrevivência dos passageiros.
<br><br>

Vamos descrever qual é o significado dos recursos dados os conjuntos de dados de treinamento e teste.
<h4>Chave de definição de variável.</h4>

- Sobrevivência

 - 0 = Não
 - 1 = Sim

- pclass (ala de ingresso)

 - 1 = 1st
 - 2 = 2nd
 - 3 = 3rd
 
- sex

- age 

- sibsp (nº de irmãos/cônjuges a bordo do Titanic)
<br>
- parch (nº de pais/filhos a bordo do Titanic)
<br>
- tickets
<br>
- fare (tarifa)
<br>
- cabin
- embarked Porto de Embarque.
 - C = Cherburgo,
 - Q = Queenstown,
 - S = Southampton
- pclass: uma proxy para status socioeconômico (SES)
<br>
<h4>É importante lembrar disso e será útil para análises posteriores.</h4>

 - 1st = Upper
 - 2nd = Middle
 - 3rd = Lower





## Parte 1. Limpando os dados.

In [ ]:
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Parece que esse conjunto de dados está bastante organizado, no entanto, antes de usar esse conjunto de dados para análise e visualização, precisamos lidar com ..
- Diferentes variáveis
- Valores nulos

## Diferentes variáveis ​​presentes nos conjuntos de dados.
- **Existem quatro tipos de variáveis**
- **Recursos numéricos**: Idade, Tarifa, SibSp e Parch
- **Recursos Categóricos**: Sexo, Embarcou, Sobreviveu e Pclass
- **Recursos alfanuméricos**: Bilhete e Cabine (Contém alfabetos e o valor numérico)
- **Recursos de texto**: Nome

**Realmente precisamos ajustar esses recursos para obter a forma desejada de dados de entrada**

In [ ]:
train.shape# Means 891 rows and 12 columns.

(891, 12)

In [ ]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Vemos que Age e Cabin têm muitos valores ausentes. Então, primeiro precisamos lidar com todos esses valores NaN.
- Como na coluna Cabine, cerca de 1\3 dos valores estão faltando. Assim, nos livramos desta coluna.
<br>

## Por que o tratamento de valores ausentes é necessário?
A falta de dados no conjunto de dados de treinamento pode reduzir o poder/ajuste de um modelo ou pode levar a um modelo tendencioso porque não analisamos corretamente o comportamento e o relacionamento com outras variáveis. Pode levar a uma previsão ou classificação errada.
- Aqui os métodos para lidar com valores ausentes.

### Imputação KNN.
------
Nesse método de imputação, os valores ausentes de um atributo são imputados usando o número determinado de atributos que são mais semelhantes ao atributo cujos valores estão ausentes.

Para mais...
<br>
**Método 1**
- [Imputação KNN](https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/)
- [Blog](https://towardsdatascience.com/the-use-of-knn-for-missing-values-cf33d935c637)

**Método 2**
- [sklearn.preprocessing.Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html)

**Usamos o Método 2, ou seja (sklearn.preprocessing.Imputer)**

Só porque é fácil de usar....

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [ ]:
Imp=SimpleImputer(missing_values=np.nan,strategy='median',axis=1)
new=Imp.fit_transform(train.Age.values.reshape(1,-1))
train['Age2']=new.T
#Lets drop the old one age Column.

TypeError: ignored

In [ ]:
train.drop('Age',axis=1,inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
train.set_index('PassengerId',inplace=True)
## get dummy variables for Column sex and embarked since they are categorical value.
train = pd.get_dummies(train, columns=["Sex"], drop_first=True)
train = pd.get_dummies(train, columns=["Embarked"],drop_first=True)


#Mapping the data.
train['Fare'] = train['Fare'].astype(int)
train.loc[train.Fare<=7.91,'Fare']=0
train.loc[(train.Fare>7.91) &(train.Fare<=14.454),'Fare']=1
train.loc[(train.Fare>14.454)&(train.Fare<=31),'Fare']=2
train.loc[(train.Fare>31),'Fare']=3

train['Age2']=train['Age2'].astype(int)
train.loc[ train['Age2'] <= 16, 'Age2']= 0
train.loc[(train['Age2'] > 16) & (train['Age2'] <= 32), 'Age2'] = 1
train.loc[(train['Age2'] > 32) & (train['Age2'] <= 48), 'Age2'] = 2
train.loc[(train['Age2'] > 48) & (train['Age2'] <= 64), 'Age2'] = 3
train.loc[train['Age2'] > 64, 'Age2'] = 4

In [ ]:
# In our data the Ticket and Cabin,Name are the base less,leds to the false prediction so Drop both of them.
train.drop(['Ticket','Cabin','Name'],axis=1,inplace=True)
train.head()
print(type(train.Age2))

h2 Parte 2.Análise de dados exploratórios /h2 .

**Análise exploratória de dados (EDA)** é uma abordagem para analisar conjuntos de dados para resumir suas principais características, geralmente com métodos visuais.


In [ ]:
train.shape
# 891 rows and 9 columns.

In [ ]:
train.Survived.value_counts()/len(train)*100
#This signifies almost 61% people in the ship died and 38% survived.

In [ ]:
train.describe()

In [ ]:
train.groupby('Survived').mean()

In [ ]:
train.groupby('Sex_male').mean()

Há alguns pontos que devem ser observados na visão geral estatística. Eles são..
- Sobre a taxa de sobrevivência, apenas 38% dos passageiros sobreviveram durante aquela tragédia.
- Sobre a taxa de sobrevivência por gênero, 74% dos passageiros do sexo feminino sobreviveram, enquanto apenas 19% dos passageiros do sexo masculino sobreviveram.

**Matriz de correlação e Heatmap**

In [ ]:
train.corr()

In [ ]:
plt.subplots(figsize = (15,8))
sns.heatmap(train.corr(), annot=True,cmap="PiYG")
plt.title("Correlations Among Features", fontsize = 20);

**Recursos de correlação positiva:**
- Tarifa e Sobrevivência: 0,26.

Existe uma correlação positiva entre a tarifa e a classificação de sobrevivência. Isso pode ser explicado dizendo que, o passageiro que pagou mais dinheiro pela passagem teve mais chances de sobreviver.

**Recursos de correlação negativa:**
- Tarifa e Pclass: -0,55
- Essa relação pode ser explicada dizendo que o passageiro de primeira classe(1) pagou mais caro que o passageiro de segunda classe(2), da mesma forma que o passageiro de segunda classe pagou mais que o passageiro de terceira classe(3).
- Gênero e Sobrevivência: -0,54
- Basicamente é a informação se o passageiro era homem ou mulher.
- Pclasse e Sobreviveu: -0,34

**Gênero e Sobrevivência**

In [ ]:
plt.subplots(figsize = (15,8))
sns.barplot(x = "Sex_male", y = "Survived", data=train, edgecolor=(0,0,0), linewidth=2)
plt.title("Survived/Non-Survived Passenger Gender Distribution", fontsize = 25)
labels = ['Female', 'Male']
plt.ylabel("% of passenger survived", fontsize = 15)
plt.xlabel("Gender",fontsize = 15)
plt.xticks(sorted(train.Sex_male.unique()), labels)

# 1 is for male and 0 is for female.

Este gráfico de barras acima mostra a distribuição de fêmeas e machos sobreviventes. O x_label mostra o sexo e o y_label mostra % de passageiros sobreviventes. Este gráfico de barras mostra que 74% dos passageiros do sexo feminino sobreviveram, enquanto apenas
19% dos passageiros do sexo masculino sobreviveram.

In [ ]:
sns.set(style='darkgrid')
plt.subplots(figsize = (15,8))
ax=sns.countplot(x='Sex_male',data=train,hue='Survived',edgecolor=(0,0,0),linewidth=2)
train.shape
## Fixing title, xlabel and ylabel
plt.title('Passenger distribution of survived vs not-survived',fontsize=25)
plt.xlabel('Gender',fontsize=15)
plt.ylabel("# of Passenger Survived", fontsize = 15)
labels = ['Female', 'Male']
#Fixing xticks.
plt.xticks(sorted(train.Survived.unique()),labels)
## Fixing legends
leg = ax.get_legend()
leg.set_title('Survived')
legs=leg.texts
legs[0].set_text('No')
legs[1].set_text('Yes')


Este gráfico de contagem mostra a distribuição real de passageiros masculinos e femininos que sobreviveram e não sobreviveram. Mostra que entre todas as fêmeas
230 sobreviveram e
70 não sobreviveram. Enquanto entre os passageiros do sexo masculino
110 sobreviveram e
480 não sobreviveram.

**Resumo**
- Como suspeitávamos, os passageiros do sexo feminino sobreviveram a uma taxa muito melhor do que os passageiros do sexo masculino.
- Parece certo, já que as mulheres e as crianças eram a prioridade.

**Pclass e Survived**

In [ ]:
train.head(4)

In [ ]:
plt.subplots(figsize = (10,10))
ax=sns.countplot(x='Pclass',hue='Survived',data=train)
plt.title("Passenger Class Distribution - Survived vs Non-Survived", fontsize = 25)
leg=ax.get_legend()
leg.set_title('Survival')
legs=leg.texts

legs[0].set_text('No')
legs[1].set_text("yes")

Portanto, parece claramente que, a sobrevivência das pessoas pertencentes à 3ª classe é muito menor.
Parece...
- 63% dos passageiros de primeira classe sobreviveram a uma tragédia titânica, enquanto
- 48% de segunda classe e
- apenas 24% dos passageiros da terceira classe sobreviveram.

In [ ]:
plt.subplots(figsize=(10,8))
sns.kdeplot(train.loc[(train['Survived'] == 0),'Pclass'],shade=True,color='r',label='Not Survived')
ax=sns.kdeplot(train.loc[(train['Survived'] == 1),'Pclass'],shade=True,color='b',label='Survived' )

labels = ['First', 'Second', 'Third']
plt.xticks(sorted(train.Pclass.unique()),labels)

Este gráfico kde é bastante auto-explicativo com todos os rótulos e cores. Algo que notei e que alguns leitores podem achar questionável é isso no enredo; os passageiros de terceira classe sobreviveram a mais de passageiros de segunda classe. É verdade, pois havia muito mais passageiros de terceira classe do que de primeira e segunda.

**Resumo**

O passageiro da primeira classe teve vantagem durante a tragédia do que os passageiros da segunda e terceira classe. Você provavelmente pode concordar comigo mais sobre isso, quando olhamos para a distribuição de passagens e coluna sobreviveu.

**Fare e Survived**

In [ ]:
plt.subplots(figsize=(15,10))

ax=sns.kdeplot(train.loc[(train['Survived'] == 0),'Fare'],color='r',shade=True,label='Not Survived')
ax=sns.kdeplot(train.loc[(train['Survived'] == 1),'Fare'],color='b',shade=True,label='Survived' )
plt.title('Fare Distribution Survived vs Non Survived',fontsize=25)
plt.ylabel('Frequency of Passenger Survived',fontsize=20)
plt.xlabel('Fare',fontsize=20)

**Age e Survived**

In [ ]:
fig,axs=plt.subplots(figsize=(10,8))
sns.set_style(style='darkgrid')
sns.kdeplot(train.loc[(train['Survived']==0),'Age2'],color='r',shade=True,label='Not Survived')
sns.kdeplot(train.loc[(train['Survived']==1),'Age2'],color='b',shade=True,label='Survived')


Não há nada fora do comum neste enredo, exceto a parte esquerda da distribuição. Mostra que

crianças e bebês eram a prioridade.

**Modelando os dados**

Vamos treinar os dados com os seguintes modelos:
- Logistic Regression
- Gaussian Naive Bayes
- Support Vector Machines
- Decision Tree Classifier
- K-Nearest Neighbors(KNN)


In [ ]:
X=train.drop('Survived',axis=1)
y=train['Survived'].astype(int)


Comparação do Classificador 

Por Comparação do Classificador, escolhemos qual modelo é o melhor para os dados fornecidos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis 
from xgboost import XGBClassifier

classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    XGBClassifier(),
    RandomForestClassifier(n_estimators=100, max_features=3),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]
    


log_cols = ["Classifier", "Accuracy"]
log= pd.DataFrame(columns=log_cols)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

sss = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
acc_dict = {}

for train_index, test_index in sss.split(X, y):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    for clf in classifiers:
        name = clf.__class__.__name__
    
        clf.fit(X_train,y_train)
        predict=clf.predict(X_test)
        acc=accuracy_score(y_test,predict)
        if name in acc_dict:
            acc_dict[name]+=acc
        else:
            acc_dict[name]=acc

In [ ]:
log['Classifier']=acc_dict.keys()
log['Accuracy']=acc_dict.values()
log.set_index([[0,1,2,3,4,5,6,7,8,9,10]])
%matplotlib inline
sns.set_color_codes("muted")
ax=plt.subplots(figsize=(10,8))
ax=sns.barplot(y='Classifier',x='Accuracy',data=log,color='b')
ax.set_xlabel('Accuracy',fontsize=20)
plt.ylabel('Classifier',fontsize=20)
plt.grid(color='r', linestyle='-', linewidth=0.5)
plt.title('Classifier Accuracy',fontsize=20)


A partir do gráfico de barras acima, podemos ver claramente que a precisão do classificador SVC é a melhor de todos os outros classificadores.

Vamos aplicar isso aos nossos dados de teste.

h2 Previsão /h2

Vamos usar o classificador SVC para prever nossos dados.

In [ ]:
train.head()

In [ ]:
classifier=SVC()
xtrain=train.iloc[:,1:]
ytrain=train.iloc[:,:1]
ytrain=ytrain.values.ravel()
classifier.fit(xtrain,ytrain)
#we need to convert the test data 

In [ ]:
testIm=Imputer(missing_values='NaN',strategy='most_frequent',axis=1)
Age1=testIm.fit_transform(test.Age.values.reshape(1,-1))
Fare2=testIm.fit_transform(test.Fare.values.reshape(1,-1))
test.drop(['Name','Cabin','Age','Ticket','Fare'],axis=1,inplace=True)
test['Age1']=Age1.T
test['Fare2']=Fare2.T
test.set_index('PassengerId',inplace=True)
#test[test.Fare.isnull()]#this will tell us which row have null so we can drop that particular row.
#test.drop([1044],axis=0,inplace=True)#drop the row which NaN .
#test.isnull().sum()

In [ ]:
## get dummy variables for Column sex and embarked since they are categorical value.
test = pd.get_dummies(test, columns=["Sex"], drop_first=True)
test = pd.get_dummies(test, columns=["Embarked"],drop_first=True)


#Mapping the data.
test['Fare2'] = test['Fare2'].astype(int)
test.loc[test.Fare2<=7.91,'Fare2']=0
test.loc[(test.Fare2>7.91) &(test.Fare2<=14.454),'Fare2']=1
test.loc[(test.Fare2>14.454)&(test.Fare2<=31),'Fare2']=2
test.loc[(test.Fare2>31),'Fare2']=3

test['Age1']=test['Age1'].astype(int)
test.loc[ test['Age1'] <= 16, 'Age1']= 0
test.loc[(test['Age1'] > 16) & (test['Age1'] <= 32), 'Age1'] = 1
test.loc[(test['Age1'] > 32) & (test['Age1'] <= 48), 'Age1'] = 2
test.loc[(test['Age1'] > 48) & (test['Age1'] <= 64), 'Age1'] = 3
test.loc[test['Age1'] > 64, 'Age1'] = 4

<h2> Resultado </h2>

O resultado final é

In [ ]:
Result=classifier.predict(test)
print(Result)
print(len(Result))